**Задачи по SQL c курса Tinkoff. Анализ данных в индустрии. Вольные слушатели 2022**

**Тинькофф Квест (легенда)**

Давайте предположим, что мы запускаем проект "Тинькофф Квест". 
* Открываем квесты во всей стране по франшизной системе, то есть, каждой локацией по
факту владеем не мы, а наши партнеры. Каждая локация принадлежит только одному
партнеру.
* Локация – это по сути помещение, при этом на одной локации может проводиться 
несколько квестов, если наш партнер сможет их все уместить, естественно с соблюдением
всех норм и правил безопасности.
* В рамках нашего проекта, партнеры в праве продавать легенды (сценарии) своих квестов
другим участникам проекта. Мы даже наняли несколько креативных агентств, которые
нам написали несколько базовых легенд для квестов.
* Игрой будем называть слот в расписании. Так вот, в рамках каждой отдельной игры, 
команде участников будет предложено пройти квест за 50 минут, кто-то справляется 
быстрее, а кто-то наоборот не успевает пройти. Бывает и такое, что игра может не 
состояться, если на нее никто не записался.
* В ходе прохождения игры, участникам будет помогать оператор.


P.S. Данная история целиком и полностью вымышленная, все совпадения с реальными 
личностями случайны :-)

[Модель хранилища данных](https://drive.google.com/file/d/1USLaTugBAPl99DCfq7N54rO-Odqnm2Ik/view?usp=sharing)

In [ ]:
# Обновление версии sqlite3 до поддерживающей оконные функции (если необходимо)

! gdown 1CDqqyKV9BvrvnuR1l8HVz0aYuM4yOTUr
! mv _sqlite3.cpython-38-x86_64-linux-gnu.so /usr/lib/python3.8/lib-dynload/
import os
os.kill(os.getpid(), 9)

"gdown" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [1]:
import sqlite3
import pandas as pd

In [2]:
sqlite3.sqlite_version

'3.40.1'

**1)** Со сколькими креативными агентствами мы работаем? Креативное агентство – это партнер без единой локации, но имеющий патент на хотя бы одну легенду.

In [3]:
con = sqlite3.connect('tinkoff_sql.db')

In [4]:
url = 'https://drive.google.com/file/d/1_pxIArWrURmECXb_y8xtZHO_JXUu4Z1R/view?usp=share_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]

In [5]:
location_df = pd.read_csv(path)
location_df.head(3)

,location_rk,partner_rk,city_nm,metro_nm
0,3854,369,Москва,Деловой центр
1,3855,371,Санкт-петербург,Пушкинская
2,3856,377,Екатеринбург,Площадь 1905 года


In [6]:
location_df.to_sql('location',con,index=False,if_exists='replace')

14

In [7]:
url = 'https://drive.google.com/file/d/1YhOLDJa-jrLmDHJUyFMwRVTbHWaY9awy/view?usp=share_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]

In [8]:
partner_df = pd.read_csv(path)
partner_df.head(3)

,partner_rk,partner_nm
0,368,Porsche
1,369,Aston Martin
2,370,Tesla


In [9]:
partner_df.to_sql('partner',con,index=False,if_exists='replace')

11

In [10]:
url = 'https://drive.google.com/file/d/1s7pyxenpTj5rflnfd6rKM-e0pJTkwGqQ/view?usp=share_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]

In [11]:
legend_df = pd.read_csv(path)
legend_df.head(3)

,legend_rk,partner_rk,legend_nm,complexity
0,28348,372,Готэм,1
1,28349,377,Иллюзия обмана,2
2,28350,372,Интерстеллар,1


In [12]:
legend_df.to_sql('legend',con,index=False,if_exists='replace')

20

In [13]:
def select(sql):
  return pd.read_sql(sql,con)

In [14]:
sql = '''SELECT 
  p.partner_rk, 
  p.partner_nm, 
  l.location_rk, 
  COUNT(leg.legend_rk) AS legends 
FROM 
  partner p 
  LEFT JOIN location l ON p.partner_rk = l.partner_rk 
  LEFT JOIN legend leg ON p.partner_rk = leg.partner_rk 
WHERE location_rk IS NULL AND legend_rk > 0
GROUP BY p.partner_rk,  p.partner_nm
HAVING legends > 0

      '''

In [15]:
select(sql)

,partner_rk,partner_nm,location_rk,legends
0,368,Porsche,None,2
1,370,Tesla,None,2
2,373,Ferrari,None,2


**Ответ: 3.** Мы работаем c 3-мя креативными агентствами: Porsche, Tesla, Ferrari.

**2)** У какого квеста (выпишите его quest_nm) разница доли состоявшихся квестов в январе и в 
феврале наибольшая по модулю? Долей считать количество состоявшихся квестов
деленное на количество заявленных. В случае наличия нескольких квестов, подходящих 
под условие, требуется вывести тот, у которого значение quest_rk больше.

In [16]:
url = 'https://drive.google.com/file/d/1qcBhbq9SR2diPmgBinSv7w3THHhpHbW_/view?usp=share_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]

In [17]:
quest_df = pd.read_csv(path)
quest_df.head(3)

,quest_rk,legend_rk,location_rk,quest_nm
0,5739,28356,3854,Человек паук - Москва
1,5740,28366,3861,Матрица - Екатеринбург
2,5741,28356,3856,Человек паук - Екатеринбург


In [18]:
quest_df.to_sql('quest',con,index=False,if_exists='replace')

35

In [19]:
url = 'https://drive.google.com/file/d/1WhTJiE6-JGqXtdYxOKunTrJ655FF-bAk/view?usp=share_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]

In [20]:
game_df = pd.read_csv(path)
game_df.head(3)

,game_rk,quest_rk,employee_rk,game_dttm,price,game_flg,finish_flg,time
0,684569,5770,28475,2018-01-23 08:15:00,2400,1,1.0,50.0
1,699126,5743,28496,2018-02-10 12:45:00,3200,1,1.0,41.0
2,717582,5768,28511,2018-02-21 21:00:00,3000,1,1.0,48.0


In [21]:
game_df.to_sql('game',con,index=False,if_exists='replace')

42480

In [22]:
sql = '''	
SELECT quest_rk,
	     quest_nm,
	     diff_mod
FROM (
	SELECT
		all_table.game_dttm AS game_dt,
		all_table.quest_rk AS quest_rk,
		all_table.quest_nm AS quest_nm,
		ABS(AVG(all_table.game_flg) - LAG(AVG(all_table.game_flg), 1) OVER w) AS diff_mod
	FROM (
				SELECT
					date(r.game_dttm,'start of month')AS game_dttm,
					l.quest_rk,
					l.quest_nm,
					r.game_flg,
					r.finish_flg
			FROM 	quest AS l JOIN game AS r
	  				ON l.quest_rk = r.quest_rk
				) AS all_table 
	GROUP BY quest_rk, quest_nm, game_dt
	WINDOW w AS (
    		PARTITION BY all_table.quest_rk
    		ORDER BY all_table.game_dttm ASC
    		ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
	ORDER BY quest_rk ASC, game_dt ASC
		 ) AS XXX
WHERE diff_mod IS NOT NULL
ORDER BY diff_mod DESC
LIMIT 3
'''

In [23]:
select(sql)

,quest_rk,quest_nm,diff_mod
0,5753,Начало - Москва,0.077573
1,5746,Зверополис - Москва,0.068452
2,5755,Крестный отец - Москва,0.066724


**Ответ: Начало - Москва**

Вариант решения без оконных функций:

In [24]:
sql = '''	
SELECT jan.quest_nm, abs(avg_jan - avg_feb) AS diff_mod
From (
  SELECT q.quest_nm, 
         avg(g.game_flg) AS avg_jan
  FROM quest q
       JOIN game g
        ON q.quest_rk = g.quest_rk
  WHERE date(g.game_dttm,'start of month') = '2018-01-01'
  GROUP BY q.quest_nm
  ORDER BY avg_jan DESC
     ) AS jan
JOIN (
  SELECT q.quest_nm, 
         avg(g.game_flg) AS avg_feb
  FROM quest q
       JOIN game g
        ON q.quest_rk = g.quest_rk
  WHERE date(g.game_dttm,'start of month') = '2018-02-01'
  GROUP BY q.quest_nm
  ORDER BY avg_feb DESC
    ) AS feb
ON jan.quest_nm = feb.quest_nm
GROUP BY jan.quest_nm
ORDER BY diff_mod desc
LIMIT 3
'''

In [25]:
select(sql)

,quest_nm,diff_mod
0,Начало - Москва,0.077573
1,Зверополис - Москва,0.068452
2,Крестный отец - Москва,0.066724


**Ответ: Начало - Москва**

**3)** В каждом городе провели награждение 3х сотрудников, в чью смену среди пройденных игр оказалось самое низкое среднее время прохождения. Выпишите фамилию девушки серебряного призера. Если таких несколько, выведите первую в алфавитном порядке

In [26]:
url = 'https://drive.google.com/file/d/1buaHYUQWDg3xbUDXtwfbm1Xteda8MeU-/view?usp=share_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]

In [27]:
employee_df = pd.read_csv(path)
employee_df.head(3)

,employee_rk,first_name,last_name,gender_cd
0,28454,Николас,Кейдж,m
1,28455,Бенедикт,Камбербэтч,m
2,28456,Лили,Рейнхарт,f


In [28]:
employee_df.to_sql('employee',con,index=False,if_exists='replace')

69

In [31]:
sql = '''
WITH cte AS (

SELECT 
	all_table.city_nm AS city,
	all_table.employee_rk	AS id_emp,
	all_table.last_name AS last_name,
	all_table.first_name AS first_name,
	all_table.gender_cd AS gender_cd,
	AVG(all_table.time) AS time_game_avg,
  RANK() OVER (PARTITION BY all_table.city_nm ORDER BY AVG(all_table.time) ASC) AS rnk
FROM (
	    SELECT *
	    FROM employee e
	         LEFT JOIN game g
	          ON e.employee_rk = g.employee_rk
	         LEFT JOIN quest q
	          ON g.quest_rk = q.quest_rk
	         LEFT JOIN location l
	          ON q.location_rk = l.location_rk
	   ) AS all_table
GROUP BY city, id_emp
ORDER BY last_name
)

SELECT * FROM cte
WHERE rnk = 2 AND gender_cd = 'f'
LIMIT 1
'''

In [32]:
select(sql)

,city,id_emp,last_name,first_name,gender_cd,time_game_avg,rnk
0,Казань,28505,Буллок,Сандра,f,39.364286,2


**Ответ: Булок Сандра**